In [75]:
import pandas as pd
import pyomo.environ as pyo
import gurobipy

In [76]:
df_number_of_apartments = pd.read_excel("Housing Problem - parameter values.xlsx", sheet_name="number_of_apartments")
df_profit_height_69 = pd.read_excel("Housing Problem - parameter values.xlsx", sheet_name="profit_height_69")
df_profit_height_120 = pd.read_excel("Housing Problem - parameter values.xlsx", sheet_name="profit_height_120")
df_profit_height_168 = pd.read_excel("Housing Problem - parameter values.xlsx", sheet_name="profit_height_168")
alphas = dict()
betas = dict()
tetas = dict()
deltas = dict()

In [77]:
alphas[("social", "corporation")] = 40
alphas[("middle", "corporation")] = 50
alphas[("free", "investor")] = 60

betas["social"] = 0.4
betas["middle"] = 0.4

tetas["social"] = 40
tetas["middle"] = 50

deltas["investor"] = 0.7

In [78]:
def HousingProblemModel(number_of_floors,
                        df_number_of_apartments,
                        df_profit,
                        alphas,
                        betas,
                        tetas,
                        deltas):
    m = pyo.ConcreteModel('HousingProblem')

    m.sectors = pyo.Set(initialize = ["social", "middle", "free"])
    m.area = pyo.Set(initialize = df_number_of_apartments['area'].drop_duplicates())
    m.owners = pyo.Set(initialize = ["corporation", "investor", "private"])
    m.floor_configurations = pyo.Set(initialize = df_number_of_apartments['configuration'].drop_duplicates())

    m.K = pyo.Param(initialize=number_of_floors)
    m.R = pyo.Param(m.area, m.floor_configurations, initialize = lambda m, j, v: df_number_of_apartments.groupby(['area', 'configuration']).sum().loc[(j, v), 'no_of_apartments'])
    m.O = pyo.Param(m.sectors, m.area, m.owners, initialize = lambda m, i, j, h: df_profit.set_index(['sector', 'area', 'owner']).loc[(i, j, h), 'profit'])
    m.alpha = pyo.Param(m.sectors, m.owners, initialize = lambda m, i, h: alphas[(i, h)] if (i, h) in alphas else 0)
    m.beta = pyo.Param(m.sectors, initialize = lambda m, i: betas[i] if i in betas else 0)
    m.teta = pyo.Param(m.sectors, initialize = lambda m, i: tetas[i] if i in tetas else 0)
    m.delta = pyo.Param(m.owners, initialize = lambda m, j: deltas[j] if j in deltas else 0)

    m.X = pyo.Var(m.floor_configurations, domain=pyo.NonNegativeIntegers)
    m.Y = pyo.Var(m.sectors, m.area, m.owners, domain=pyo.NonNegativeIntegers)
    m.W = pyo.Var(m.floor_configurations, m.owners, domain=pyo.NonNegativeIntegers)
    m.Z = pyo.Var(m.area, m.owners, domain=pyo.NonNegativeIntegers)

    @m.Objective(sense=pyo.maximize)
    def obj(m):
          return pyo.quicksum(m.O[i, j, h] * m.Y[i, j, h] for i in m.sectors for j in m.area for h in m.owners)

    @m.Constraint()
    def total_number_of_floors_constraint(m):
        return pyo.quicksum(m.X[v] for v in m.floor_configurations) == m.K

    @m.Constraint(m.area)
    def total_apartments_consistency(m, j):
        return pyo.quicksum(m.R[j, v] * m.X[v] for v in m.floor_configurations) == pyo.quicksum(m.Y[i, j, h] for i in m.sectors for h in m.owners)

    @m.Constraint(m.floor_configurations)
    def owner_floor_configuration_constraint(m, v):
        return pyo.quicksum(m.W[v, h] for h in m.owners) == m.X[v]

    @m.Constraint(m.area, m.owners)
    def no_floor_with_multiple_owners_constraint(m, j, h):
        return m.Z[j, h] == pyo.quicksum(m.W[v, h] * m.R[j, v] for v in m.floor_configurations)

    @m.Constraint(m.area, m.owners)
    def apartments_by_owner_and_area_constraint(m, j, h):
        return m.Z[j, h] == pyo.quicksum(m.Y[i, j, h] for i in m.sectors)

    @m.Constraint(m.sectors)
    def minimal_percentage_of_sector_in_total_program_constraint(m, i):
        return pyo.quicksum(m.Y[i, j, h] for j in m.area for h in m.owners) >= m.beta[i] * pyo.quicksum(m.Y[l, j, h] for l in m.sectors for j in m.area for h in m.owners)

    @m.Constraint(m.sectors)
    def minimal_average_floor_area_of_apartments_in_sector_constraint(m, i):
        return pyo.quicksum(j * m.Y[i, j, h] for j in m.area for h in m.owners) >= m.teta[i] * pyo.quicksum(m.Y[i, j, h] for j in m.area for h in m.owners)

    @m.Constraint(m.sectors, m.area, m.owners)
    def minimal_floor_area_for_sector_and_owner_constraint(m, i, j, h):
        if j < m.alpha[i, h]:
          return m.Y[i, j, h] == 0
        else:
          return pyo.Constraint.Skip

    @m.Constraint(m.area)
    def no_free_sector_apartments_for_housing_corporation_constraint(m, j):
        return m.Y["free", j, "corporation"] == 0

    @m.Constraint(m.owners)
    def sector_ownership_requirement_constraint(m, h):
        return pyo.quicksum(m.Y[i, j, h] for i in m.sectors for j in m.area) >= m.delta[h] * pyo.quicksum(m.Y[i, j, p] for i in m.sectors for j in m.area for p in m.owners)

    return m

In [79]:
model_23 = HousingProblemModel(23, df_number_of_apartments, df_profit_height_69, alphas, betas, tetas, deltas)

In [80]:
solver = pyo.SolverFactory('gurobi')
results_23 = solver.solve(model_23)

In [81]:
print(f"objective value is {pyo.value(model_23.obj)}")

objective value is 17483910.86302152


In [82]:
for v in model_23.component_objects(pyo.Var, active=True):
    print(f"Variable {v.name}")
    for index in v:
      if v[index].value > 0:
        print(f"  {index}: {v[index].value}")

Variable X
  aa: 11.0
  ac: 1.0
  cc: 1.0
  cd: 1.0
  ee: 9.0
Variable Y
  ('social', 36, 'investor'): 46.0
  ('social', 42, 'investor'): 27.0
  ('social', 60, 'corporation'): 2.0
  ('social', 70, 'corporation'): 1.0
  ('social', 71, 'corporation'): 2.0
  ('middle', 42, 'investor'): 19.0
  ('middle', 48, 'investor'): 46.0
  ('middle', 60, 'investor'): 6.0
  ('middle', 71, 'investor'): 5.0
  ('middle', 96, 'corporation'): 2.0
  ('free', 71, 'investor'): 1.0
  ('free', 131, 'private'): 36.0
Variable W
  ('aa', 'investor'): 11.0
  ('ac', 'investor'): 1.0
  ('cc', 'investor'): 1.0
  ('cd', 'corporation'): 1.0
  ('ee', 'private'): 9.0
Variable Z
  (36, 'investor'): 46.0
  (42, 'investor'): 46.0
  (48, 'investor'): 46.0
  (60, 'corporation'): 2.0
  (60, 'investor'): 6.0
  (70, 'corporation'): 1.0
  (71, 'corporation'): 2.0
  (71, 'investor'): 6.0
  (96, 'corporation'): 2.0
  (131, 'private'): 36.0


In [85]:
model_40 = HousingProblemModel(40, df_number_of_apartments, df_profit_height_120, alphas, betas, tetas, deltas)

In [86]:
solver = pyo.SolverFactory('gurobi')
results_40 = solver.solve(model_40)

In [87]:
print(f"objective value is {pyo.value(model_40.obj)}")

objective value is 29622229.854210615


In [88]:
for v in model_40.component_objects(pyo.Var, active=True):
    print(f"Variable {v.name}")
    for index in v:
      if v[index].value > 0:
        print(f"  {index}: {v[index].value}")

Variable X
  aa: 19.0
  bc: 2.0
  cc: 3.0
  ee: 16.0
Variable Y
  ('social', 36, 'investor'): 76.0
  ('social', 42, 'investor'): 50.0
  ('social', 60, 'corporation'): 4.0
  ('social', 71, 'corporation'): 4.0
  ('middle', 42, 'investor'): 30.0
  ('middle', 48, 'investor'): 76.0
  ('middle', 52, 'investor'): 4.0
  ('middle', 60, 'investor'): 10.0
  ('middle', 68, 'investor'): 2.0
  ('middle', 71, 'investor'): 12.0
  ('free', 60, 'investor'): 2.0
  ('free', 131, 'private'): 64.0
Variable W
  ('aa', 'investor'): 19.0
  ('bc', 'investor'): 2.0
  ('cc', 'corporation'): 1.0
  ('cc', 'investor'): 2.0
  ('ee', 'private'): 16.0
Variable Z
  (36, 'investor'): 76.0
  (42, 'investor'): 80.0
  (48, 'investor'): 76.0
  (52, 'investor'): 4.0
  (60, 'corporation'): 4.0
  (60, 'investor'): 12.0
  (68, 'investor'): 2.0
  (71, 'corporation'): 4.0
  (71, 'investor'): 12.0
  (131, 'private'): 64.0


In [89]:
model_56 = HousingProblemModel(56, df_number_of_apartments, df_profit_height_168, alphas, betas, tetas, deltas)

In [90]:
solver = pyo.SolverFactory('gurobi')
results_56 = solver.solve(model_56)

In [91]:
print(f"objective value is {pyo.value(model_56.obj)}")

objective value is 39187428.96979929


In [92]:
for v in model_56.component_objects(pyo.Var, active=True):
    print(f"Variable {v.name}")
    for index in v:
      if v[index].value > 0:
        print(f"  {index}: {v[index].value}")

Variable X
  aa: 27.0
  ac: 1.0
  cc: 4.0
  ce: 1.0
  ee: 23.0
Variable Y
  ('social', 36, 'investor'): 110.0
  ('social', 42, 'investor'): 63.0
  ('social', 48, 'investor'): 1.0
  ('social', 60, 'corporation'): 6.0
  ('social', 71, 'corporation'): 6.0
  ('middle', 42, 'investor'): 47.0
  ('middle', 48, 'investor'): 109.0
  ('middle', 60, 'investor'): 14.0
  ('middle', 71, 'investor'): 14.0
  ('middle', 131, 'corporation'): 2.0
  ('free', 131, 'private'): 92.0
Variable W
  ('aa', 'investor'): 27.0
  ('ac', 'investor'): 1.0
  ('cc', 'corporation'): 1.0
  ('cc', 'investor'): 3.0
  ('ce', 'corporation'): 1.0
  ('ee', 'private'): 23.0
Variable Z
  (36, 'investor'): 110.0
  (42, 'investor'): 110.0
  (48, 'investor'): 110.0
  (60, 'corporation'): 6.0
  (60, 'investor'): 14.0
  (71, 'corporation'): 6.0
  (71, 'investor'): 14.0
  (131, 'corporation'): 2.0
  (131, 'private'): 92.0
